In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import time
import random

def check_for_error_message(driver, error_message):
    elements = driver.find_elements(By.TAG_NAME, 'p') + driver.find_elements(By.TAG_NAME, 'li')
    for element in elements:
        if error_message in element.text:
            return True
    return False

# Read phone numbers from file and remove country code
with open('USTMphonenumber.txt', 'r') as file:
    phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file][130:135]

In [4]:

# Initialize WebDriver
options = webdriver.ChromeOptions()
options.headless = True
driver = webdriver.Chrome(options=options)

driver.get("https://prepaid.t-mobile.com/direct-to-account?icid=MGPR_TMO_C_CUSTSUPT_PE7I1K07DR3HZU5OA19473")

results = []
i = 0
try:
    for phone_number in phone_numbers:
        print(i)
        i+=1
        # Wait for the first phone number input field and enter the phone number one character at a time
        primary_phone_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "tmo-input-default-17"))
        )
        primary_phone_input.clear()
        for digit in phone_number:
            primary_phone_input.send_keys(digit)
            time.sleep(time.sleep(random.uniform(0.3, 1)))  # Mimic human typing

        # Wait a moment for possible error messages to appear
        time.sleep(time.sleep(random.uniform(2, 5)))

        # # Check if the tourist plan error message is displayed after the first input
        # if driver.find_elements(By.CSS_SELECTOR, "p.m-b-5.xs-m-b-13"):
        #     results.append((phone_number, 0))  # Tourist plan error
        #     continue  # Skip this iteration and continue with the next phone number

        # Check for the confirmation checkmark
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "i.fa.fa-check.def.active.m-r-sm"))
        )

        # Enter the phone number again in the second input field one character at a time
        confirm_phone_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "tmo-input-default-18"))
        )
        confirm_phone_input.clear()
        for digit in phone_number:
            confirm_phone_input.send_keys(digit)
            time.sleep(time.sleep(random.uniform(0.3, 1)))  # Mimic human typing

        # Click an empty space 
        empty_space = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.TAG_NAME, "header"))
        )
        empty_space.click()

        # Wait for possible error messages to appear after second input
        time.sleep(time.sleep(random.uniform(2, 5)))

        # Check for all possible error messages
        if check_for_error_message(driver, "Refills and Payments are not allowed on the Tourist rate plan"):
            error_code = 3
        elif check_for_error_message(driver, "Mobile number is invalid. Please try again."):
            error_code = 2
        elif check_for_error_message(driver, "Mobile number is invalid."):
            error_code = 1
        else:
            error_code = 0  # No error found

        results.append((phone_number, error_code))

        # Refresh the page to reset state for next input
        time.sleep(time.sleep(random.uniform(3, 10)))
        #driver.refresh()

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    driver.quit()


0
An error occurred: an integer is required (got type NoneType)


In [42]:
import os

csv_file_path = 'ustm_query_result.csv'

df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
df_results


,Phone Number,Error Code
0,2038000635,1
1,4158467061,1
2,3307084296,2
3,3106546516,1
4,8315159473,1
5,7089851055,2
6,5136558875,3
7,9137069682,2
8,5129982913,3
9,3619343787,1


In [70]:
import os

csv_file_path = 'ustm_query_result.csv'

df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
if os.path.exists(csv_file_path):
    df_existing = pd.read_csv(csv_file_path)
    
    last_index = df_existing.index[-1] if not df_existing.empty else -1

    df_results.index = range(last_index + 1, last_index + 1 + len(df_results))
    
    # Append the new results to the existing DataFrame
    df_final = df_existing.append(df_results)
    df_final = df_final.iloc[last_index + 1:,1:]
else:
    df_final = df_results
    df_final.index = range(len(df_final))

df_final


C:\Users\ericluo\AppData\Local\Temp\ipykernel_24792\3829665517.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_existing.append(df_results)


,Phone Number,Error Code
135,3365755149,1
136,8189740756,2
137,8457647765,2
138,8152509998,2
139,4013017288,2


In [71]:
# Output to CSV, without overwriting existing data and with continuous indexing
df_final.to_csv(csv_file_path, index_label='Index', mode='w' if not os.path.exists(csv_file_path) else 'a', header=not os.path.exists(csv_file_path))

print(f"Results have been appended to {csv_file_path}")


Results have been appended to ustm_query_result.csv
